In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Data

In [24]:
# Load the training dataframe
train_data = pd.read_csv("data/train.csv")

# Preview the train dataframe
display(train_data.head(10))

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
5,IWHA,0
6,GEFV,0
7,KARS,0
8,IMGW,0
9,KYRK,0


In [23]:
# Split sequences into characters
first = []
second = []
third = []
fourth = []

for _, row in data.iterrows():
    ch_1, ch_2, ch_3, ch_4 = row["Sequence"]
    
    first.append(ch_1)
    second.append(ch_2)
    third.append(ch_3)
    fourth.append(ch_4)

# Create X_df
X_df_dict = {
    "first": first,
    "second": second,
    "third": third,
    "fourth": fourth
}
X_df = pd.DataFrame(X_df_dict)

D K W L DKWL


Exception: 

## OneHot Encoding

In [45]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_df)

display(enc.categories_)

# Ensure the onehots are the same for each letter
assert((enc.categories_[0] == enc.categories_[1]).all() and (enc.categories_[1] == enc.categories_[2]).all() and (enc.categories_[2] == enc.categories_[3]).all())

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='ignore', sparse=True)

# Training

In [61]:
# Create training np arrays
X = enc.transform(data_split).toarray()
y = train_data["Active"].values

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

X=enc_data
y=np.array(data["Active"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [53]:
clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(15,), batch_size=128, verbose=True), verbose=True)
clf.fit(X_train, y_train)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.2s
[Pipeline] ..... (step 2 of 2) Processing mlpclassifier, total=  26.6s


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlpclassifier',
                 MLPClassifier(activation='relu', alpha=0.0001, batch_size=128,
                               beta_1=0.9, beta_2=0.999, early_stopping=False,
                               epsilon=1e-08, hidden_layer_sizes=(15,),
                               learning_rate='constant',
                               learning_rate_init=0.001, max_fun=15000,
                               max_iter=200, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='adam',
                               tol=0.0001, validation_fraction=0.1,
                               verbose=False, warm_start=False))],
         verbose=True)

In [54]:
from sklearn.metrics import f1_score
y_pred=clf.predict(X_val)
f1_score(y_val, y_pred)

0.864994026284349

In [55]:
test_data=pd.read_csv("data/test.csv")

In [56]:
first=[]
second=[]
third=[]
fourth=[]
for index,row in test_data.iterrows():
    first.append(split(row[0])[0])
    second.append(split(row[0])[1])
    third.append(split(row[0])[2])
    fourth.append(split(row[0])[3])

In [57]:
test_data_split=pd.DataFrame()
test_data_split["First"]=first
test_data_split["Second"]=second
test_data_split["Third"]=third
test_data_split["Fourth"]=fourth

In [58]:
enc_test_data=enc.transform(test_data_split).toarray()

In [59]:
y_test=clf.predict(enc_test_data)

In [60]:
np.savetxt("submission.txt", y_test, fmt="%d")